In [1]:
!pip3 install qiskit pylab-sdk

     |████████████████████████████████| 6.7MB 4.3MB/s 
     |████████████████████████████████| 23.3MB 93kB/s 
     |████████████████████████████████| 163kB 41.4MB/s 
     |████████████████████████████████| 184kB 47.7MB/s 
     |████████████████████████████████| 1.9MB 41.9MB/s 
     |████████████████████████████████| 51kB 6.3MB/s 
     |████████████████████████████████| 962kB 48.2MB/s 
     |████████████████████████████████| 5.8MB 45.0MB/s 
     |████████████████████████████████| 51kB 5.6MB/s 
     |████████████████████████████████| 296kB 48.0MB/s 
     |████████████████████████████████| 71kB 8.9MB/s 
     |████████████████████████████████| 26.1MB 1.5MB/s 
     |████████████████████████████████| 583kB 43.1MB/s 
     |████████████████████████████████| 2.7MB 34.4MB/s 
  Created wheel for qiskit: filename=qiskit-0.19.6-cp36-none-any.whl size=3019 sha256=bc9b2e4a918b53ebe39c1b14c833f68612dcce601ee09d101460e497ba4a1c99
  Stored in directory: /root/.cache/pip/wheels/35/45/55/8b82d355edf738ebd

In [2]:
import numpy as np
import pylab
import copy
from qiskit import BasicAer, Aer, IBMQ
from qiskit.aqua import aqua_globals, QuantumInstance
from qiskit.aqua.algorithms import VQE, ExactEigensolver
from qiskit.aqua.components.optimizers import COBYLA, L_BFGS_B, SLSQP, SPSA
from qiskit.aqua.components.optimizers import AQGD, ADAM
from qiskit.chemistry.components.initial_states import HartreeFock
from qiskit.chemistry.components.variational_forms import UCCSD
from qiskit.aqua.components.variational_forms import RY, RYRZ, SwapRZ
from qiskit.chemistry.drivers import PySCFDriver
from qiskit.chemistry.core import Hamiltonian, QubitMappingType

from qiskit.providers.aer import noise

IBMQ.save_account('72c728803a4e599bd5029744cf7b087d1e2e63661861ce9568229755f30769ec71dafded5d2f01ffa2d384d75b1fac07ca5d5ccad905b0f661790c28cbf3e3fb')
IBMQ.load_account()

<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>

In [3]:
import warnings
warnings.filterwarnings('ignore')

backend = Aer.get_backend('qasm_simulator')

provider = IBMQ.get_provider(hub='ibm-q')
#Define our noise model based on the ibmq_essex chip
chip_name = 'ibmq_essex'
device = provider.get_backend(chip_name)
coupling_map = device.configuration().coupling_map
noise_model = noise.device.basic_device_noise_model(device.properties())
basis_gates = noise_model.basis_gates
NUM_SHOTS = 10000

def AMSGRAD(*args, **kwargs):
    kwargs['amsgrad'] = True
    Args = args
    Kwargs = kwargs
    return ADAM(*args, **kwargs)

Basic VQE vs ExactEigensolver
----

    molecule = 'H .0 .0 1.6; Li .0 .0 .0'
    algorithms = ['VQE', 'ExactEigensolver']


    for j in range(len(algorithms)):   
            print("\n\n", algorithms[j], "\n", '='*25)
            driver = PySCFDriver(molecule, basis='sto3g')
            qmolecule = driver.run()
            operator =  Hamiltonian(qubit_mapping=QubitMappingType.PARITY,
                                    two_qubit_reduction=True, freeze_core=True,
                                    orbital_reduction=[-3, -2])
            qubit_op, aux_ops = operator.run(qmolecule)
            if algorithms[j] == 'ExactEigensolver':
                result = ExactEigensolver(qubit_op, aux_operators=aux_ops).run()
                lines, result = operator.process_algorithm_result(result)
                # print(result)
                [print(x) for x in lines]
            else:
            # optimizer = COBYLA(maxiter=1000)
            # optimizer = L_BFGS_B(maxiter=1000)
            # optimizer = SLSQP(maxiter=1000)
            # optimizer = SPSA(max_trials=1000)

            # optimizer = AQGD(maxiter=1000, momentum=0.25)
            # optimizer = ADAM(maxiter=1000, lr=0.001, amsgrad=False)
            optimizer = AMSGRAD(maxiter=1000)

            initial_state = HartreeFock(operator.molecule_info['num_orbitals'],
                                        operator.molecule_info['num_particles'],
                                        qubit_mapping=operator._qubit_mapping,
                                        two_qubit_reduction=operator._two_qubit_reduction)
            
            var_form = UCCSD(num_orbitals=operator.molecule_info['num_orbitals'],
                            num_particles=operator.molecule_info['num_particles'],
                            initial_state=initial_state,
                            qubit_mapping=operator._qubit_mapping,
                            two_qubit_reduction=operator._two_qubit_reduction)
            
            algo = VQE(qubit_op, var_form, optimizer)
            result = algo.run(QuantumInstance(BasicAer.get_backend('statevector_simulator')))
                
            result = operator.process_algorithm_result(result)
            print(result)

In [10]:
def SolveEigen(
    solver = "ExactEigensolver", 
    optimizer = "COBYLA", 
    var_form = "UCCSD",
    q_mapping = QubitMappingType.PARITY,
    molecule = 'H .0 .0 .0; Li .0 .0 .0',
    depth = 3, 
    iter = 1000):

        counts = []
        values = []
        params = []
        deviation = []

        def store_intermediate_result(eval_count, parameters, mean, std):
            counts.append(eval_count)
            values.append(mean)
            params.append(parameters)
            deviation.append(std)


        print(
            "="*50,
            f"\nUsing Solver : {solver}, \nOptimizer : {optimizer} \nVariational Form : {var_form} \nQ-mappin : {str(q_mapping)} \n\t on Molecule ({molecule})\n",
            '='*50, "\n"*3
        )
        driver = PySCFDriver(molecule, basis='sto3g')
        qmolecule = driver.run()

        if q_mapping==QubitMappingType.PARITY:
          operator =  Hamiltonian(qubit_mapping=q_mapping,
                                  two_qubit_reduction=True, freeze_core=True,
                                  orbital_reduction=[-3, -2])
        elif q_mapping==QubitMappingType.BRAVYI_KITAEV:
          operator =  Hamiltonian(qubit_mapping=q_mapping,
                                  two_qubit_reduction=False, freeze_core=True,
                                  orbital_reduction=[-3, -2])
        elif q_mapping==QubitMappingType.JORDAN_WIGNER:
          operator =  Hamiltonian(qubit_mapping=q_mapping,
                                  two_qubit_reduction=False, freeze_core=True,
                                  orbital_reduction=[-3, -2])
        qubit_op, aux_ops = operator.run(qmolecule)


        num_qubits = qubit_op.num_qubits

        return_text = f"{solver}|{optimizer}|{var_form}|{q_mapping}" 

        if solver == "ExactEigensolver":
            result = ExactEigensolver(qubit_op, aux_operators=aux_ops).run()
            lines, result = operator.process_algorithm_result(result)
            # print(result)
            [print(x) for x in lines]
            print("\n"*3)
            return return_text, result, (counts, values, params, deviation)
        elif solver == "QVE":
            optimizer = COBYLA(maxiter=iter)
            
            if optimizer == "COBYLA":
                optimizer = COBYLA(maxiter=iter)
            elif optimizer == "L_BFGS_B":
                optimizer = L_BFGS_B(maxiter=iter)
            elif optimizer == "SLSQP":
                optimizer = SLSQP(maxiter=iter)
            elif optimizer == "SPSA":
                optimizer = SPSA(max_trials=iter)

            # Some Other Intresting Optimizers...
            elif optimizer == "AQGD":
                optimizer = AQGD(maxiter=iter, momentum=0.25)
            elif optimizer == "ADAM":
                optimizer = ADAM(maxiter=iter, lr=0.001, amsgrad=False)
            elif optimizer == "AMSGRAD":
                optimizer = AMSGRAD(maxiter=iter, lr=0.001)

            initial_state = HartreeFock(operator.molecule_info['num_orbitals'],
                                      operator.molecule_info['num_particles'],
                                      qubit_mapping=operator._qubit_mapping,
                                      two_qubit_reduction=operator._two_qubit_reduction)
            '''
          
            var_form = UCCSD(num_orbitals=operator.molecule_info['num_orbitals'],
                          num_particles=operator.molecule_info['num_particles'],
                          initial_state=initial_state,
                          qubit_mapping=operator._qubit_mapping,
                          two_qubit_reduction=operator._two_qubit_reduction)
            '''
            if var_form == "UCCSD":
                var_form = UCCSD(
                          num_orbitals=operator.molecule_info['num_orbitals'], 
                          num_particles=operator.molecule_info['num_particles'],
                          initial_state=initial_state,
                          qubit_mapping=operator._qubit_mapping,
                          two_qubit_reduction=operator._two_qubit_reduction)
            elif var_form == "RY":
                var_form = RY(num_qubits=num_qubits,
                          depth=depth)
            elif var_form == "RYRZ":
                var_form = RYRZ(num_qubits=num_qubits,
                          depth=depth,
                          initial_state=initial_state)
            elif var_form == "SwapRZ":
                var_form = SwapRZ(num_qubits=num_qubits,
                          depth=depth,
                          initial_state=initial_state)
          
            algo = VQE(qubit_op, var_form, optimizer, callback=store_intermediate_result)
            # result = algo.run(QuantumInstance(backend, noise_model=noise_model))
            result = algo.run(QuantumInstance(BasicAer.get_backend('statevector_simulator')))
              
            result = operator.process_algorithm_result(result)
            print(result)
            print("\n"*3)
            return return_text, result, (counts, values, params, deviation)



# Examples
---
```
solve1 = SolveEigen(solver = "ExactEigensolver", molecule = 'H .0 .0 1.6; Li .0 .0 .0')
solve2 = SolveEigen(solver = "QVE", optimizer = "COBYLA", var_form = "RY", molecule = 'H .0 .0 1.6; Li .0 .0 .0')
solve3 = SolveEigen(solver = "QVE", optimizer = "COBYLA", var_form = "RYRZ", molecule = 'H .0 .0 1.6; Li .0 .0 .0')
solve4 = SolveEigen(solver = "QVE", optimizer = "COBYLA", var_form = "SwapRZ", molecule = 'H .0 .0 1.6; Li .0 .0 .0')
```

In [ ]:
molecule = 'H .0 .0 1.6; Li .0 .0 .0'

# optimizer = ["COBYLA" , "L_BFGS_B", "SLSQP", "SPSA", "AQGD", "ADAM", "AMSGRAD"]
# var_form = ["UCCSD", "RY", "RYRZ", "SwapRZ"]
# q_mapping = [QubitMappingType.PARITY, QubitMappingType.BRAVYI_KITAEV, QubitMappingType.JORDAN_WIGNER]


optimizer = ["L_BFGS_B"]
var_form = ["UCCSD", "RY", "RYRZ", "SwapRZ"]
q_mapping = [QubitMappingType.PARITY, QubitMappingType.BRAVYI_KITAEV, QubitMappingType.JORDAN_WIGNER]



optimization_matrix = {}

intermediate_result = {}

# ExactEigensolver
k, v, _ = SolveEigen(solver = "ExactEigensolver", molecule=molecule)
optimization_matrix[k] = v

# QVEs
for opt in optimizer:
    for vform in var_form:
        for mapping in q_mapping:
            k, v, ir = SolveEigen(solver="QVE", optimizer=opt, var_form=vform, q_mapping=mapping, molecule=molecule, iter=50)
            optimization_matrix[k] = v
            intermediate_result[k] = ir
    

# print(f"{ir}")

Using Solver : ExactEigensolver, 
Optimizer : COBYLA 
Variational Form : UCCSD 
Q-mappin : QubitMappingType.PARITY 
	 on Molecule (H .0 .0 1.6; Li .0 .0 .0)



=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -8.873279314506
  - computed part:      -1.077059745735
  - frozen energy part: -7.796219568771
  - particle hole part: 0.0
~ Nuclear repulsion energy (Hartree): 0.992207270475
> Total ground state energy (Hartree): -7.881072044031
  Measured:: Num particles: 2.000, S: 0.000, M: 0.00000
 
=== DIPOLE MOMENT ===
 
* Electronic dipole moment (a.u.): [0.0  0.0  4.85967927]
  - computed part:      [0.0  0.0  4.86373119]
  - frozen energy part: [0.0  0.0  -0.00405191]
  - particle hole part: [0.0  0.0  0.0]
~ Nuclear dipole moment (a.u.): [0.0  0.0  3.0235618]
> Dipole moment (a.u.): [0.0  0.0  -1.83611747]  Total: 1.83611747
               (debye): [0.0  0.0  -4.66694466]  Total: 4.66694466




Using Solver : QVE, 
Optimizer : L_BFGS_B 
Variational Form : UCCSD

In [8]:
import pickle
def save_object(obj, filename):
    with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)

save_object(optimization_matrix, "L_BFGS_B_optimization_matrix.pkl")
save_object(intermediate_result, "L_BFGS_B_intermediate_result.pkl")

# optimization_matrix= None

# with open('optimization_matrix.pkl', 'rb') as input:
#     optimization_matrix = pickle.load(input)
# optimization_matrix

In [ ]:
all_solvers = [k for k in optimization_matrix]
solver_energy = {}
solver_energy[all_solvers[0]] = optimization_matrix[all_solvers[0]]['energies'][0]
for i in range(1, len(all_solvers)):
    cur = all_solvers[i]
    solver_energy[cur] = optimization_matrix[cur].energy


In [ ]:
solver_energy

In [ ]:
import operator

d = dict(solver_energy)
sorted_d = dict(sorted(d.items(), key=operator.itemgetter(1)))
print(sorted_d)

# Create full graph for all

In [ ]:
# distances = [0.3, 0.5, 0.7, 0.9, 1.1, 1.3, 1.5, 1.7, 1.9, 2.1, 2.3, 2.5, 2.7, 2.9, 3.1, 3.3, 3.5, 3.7, 3.9, 4.1, 4.3, 4.5, 4.7, 4.9]
distances = [0.3, 0.9, 1.5, 2.1, 2.7, 3.3, 3.9, 4.5]
optimizer = ["COBYLA", "L_BFGS_B", "SLSQP", "SPSA", "AQGD", "ADAM", "AMSGRAD"]
var_form = ["UCCSD", "RY", "RYRZ", "SwapRZ"]
q_mapping = [QubitMappingType.PARITY, QubitMappingType.BRAVYI_KITAEV, QubitMappingType.JORDAN_WIGNER]

optimization_matrix = {}


optimization_matrix["ExactEigensolver|COBYLA|UCCSD|QubitMappingType.PARITY"] = []
for opt in optimizer:
    for vform in var_form:
        for mapping in q_mapping:
            optimization_matrix[f"QVE|{opt}|{vform}|{mapping}"] = []

# ExactEigensolver
for distance in distances:
    k, v = SolveEigen(solver = "ExactEigensolver", molecule= f'H .0 .0 {distance}; Li .0 .0 .0')
    optimization_matrix[k] += [v]

# QVEs
for distance in distances:
    print("\n"*10 ,  f"Working on distance {distance}\n")
    for opt in optimizer:
        for vform in var_form:
            for mapping in q_mapping:
                k, v = SolveEigen(solver="QVE", optimizer=opt, var_form=vform, q_mapping=mapping, molecule= f'H .0 .0 {distance}; Li .0 .0 .0', iter=50)
                optimization_matrix[k] += [v]
    

# Ploting the graphs

In [ ]:
k, solve = SolveEigen(solver = "QVE", optimizer = "L_BFGS_B", var_form = "RY", q_mapping=QubitMappingType.PARITY, iter=100, molecule = f'H .0 .0 1.6; Li .0 .0 .0')
[solve.energy]

Using Solver : QVE, 
Optimizer : COBYLA 
Variational Form : UCCSD 
	 on Molecule (H .0 .0 1.6; Li .0 .0 .0)



=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -8.873248594084
  - computed part:      -1.077029025312
  - frozen energy part: -7.796219568771
  - particle hole part: 0.0
~ Nuclear repulsion energy (Hartree): 0.992207270475
> Total ground state energy (Hartree): -7.881041323609






[-7.881041323608712]

In [9]:
# Array with all of the energies that are calculated
exact_energies = []
vqe_energies = []
# These are the distances I want to test
distances = [0.3, 0.5, 0.7, 0.9, 1.1, 1.3, 1.5, 1.7, 1.9, 2.1, 2.3, 2.5, 2.7, 2.9, 3.1, 3.3, 3.5, 3.7, 3.9, 4.1, 4.3, 4.5, 4.7, 4.9]
for distance in distances:
    k, solve = SolveEigen(solver = "QVE", optimizer = "COBYLA", var_form = "UCCSD", q_mapping=QubitMappingType.PARITY, iter=10, molecule = f'H .0 .0 {distance}; Li .0 .0 .0')
    vqe_energies += [solve.energy]
for distance in distances:
    k, solve = SolveEigen(solver = "ExactEigensolver", molecule = f'H .0 .0 {distance}; Li .0 .0 .0')
    exact_energies += [solve['energies'][0]]
    print(exact_energies)

In [ ]:
import matplotlib.pyplot as plt
from scipy.interpolate import make_interp_spline, BSpline
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [ ]:
# Draws a plot of the ground energy of the H2 molecule based on the atomic distace

plt.plot(distances, exact_energies, label = 'Exact Energy')
plt.plot(distances, vqe_energies, label = 'VQE Energy')
plt.xlabel('Atomic distace (Angstrom)')
plt.ylabel('Energy (Hartee Energy)')
plt.title('Ground Energy of LiH')
plt.legend()
plt.show()